<h1>GA4GH Disease</h1>
<p>This notebook demonstrates how to use the oncopacket Python package to create GA4GH Disease messages from Cancer Data Aggregator (CDA) data.
We first extract data about a subjects in a CDA cohort and then use the package to create the Individual messages.</p>
<p>The data is extracted from the <tt>subject</tt> table of CDA.</p>

In [1]:
import oncopacket
from collections import defaultdict

ModuleNotFoundError: No module named 'oncopacket'